In [1]:
import requests
import json
from datetime import datetime
import time
import geocoder

In [ ]:
class ISS():

    def __init__ (self):

        self.welcome = "Ladies and Gentlemen, welcome aboard the ISS!"
        self.response = requests.get("http://api.open-notify.org/iss-now.json").json() 
        self.iss_time = datetime.fromtimestamp(self.response['timestamp']).strftime('%H:%M:%S')


    def verify(self):

        if self.response['message'] ==  'success':
           return "Sucess! We're floating in space!"

        else:
           return "Ladies and Gentlemen, we're out of orbit!"

    def whos_aboard (self):
        return [names['name'] for names in requests.get("http://api.open-notify.org/astros.json").json()['people'] if names['craft'] == 'ISS']

    def position (self):
        self.iss_latitude = self.response['iss_position']['latitude']
        self.iss_longitude = self.response['iss_position']['longitude']

        return self.iss_latitude , self.iss_longitude 

In [27]:
class Where_am_I():

    def __init__(self, nome):
        self.nome = nome
        self.current_time = datetime.now().strftime('%H:%M:%S')

    def current_location_name (self):
        return geocoder.ip('me')
    
    def coordinates(self):
        self.latitude = geocoder.ip('me').lat
        self.longitude = geocoder.ip('me').lng

        return self.latitude , self. longitude